统计分析每个子数据集问答对的token长度

In [2]:
import json


path = './type1_113.json'
with open(path, 'r', encoding='utf-8') as f:
    datas = json.load(f)

In [3]:
prompt_list = [data['prompt'] for data in datas]
target_new_list = [data['target_new'] for data in datas]

In [4]:
from transformers import AutoTokenizer

# Load the tokenizer
model_path = '/share/huggingface/'
model_name = 'Qwen-1_8B-Chat'
tokenizer = AutoTokenizer.from_pretrained(model_path + model_name, trust_remote_code=True)

# Print some tokens from the tokenizer's vocabulary to identify a suitable PAD/EOS token
for i, token in enumerate(tokenizer.get_vocab()):
    if i < 10:  # Print the first 10 tokens as a sample
        print(f"Token ID: {i}, Token: {tokenizer.convert_ids_to_tokens(i)}")


Token ID: 0, Token: b'!'
Token ID: 1, Token: b'"'
Token ID: 2, Token: b'#'
Token ID: 3, Token: b'$'
Token ID: 4, Token: b'%'
Token ID: 5, Token: b'&'
Token ID: 6, Token: b"'"
Token ID: 7, Token: b'('
Token ID: 8, Token: b')'
Token ID: 9, Token: b'*'


In [6]:
len(tokenizer.get_vocab())

151643

In [9]:
list(tokenizer.get_vocab().keys())[-1]

b'\xe2\xbd\x97'

In [10]:
tokenizer.convert_ids_to_tokens(151643)

'<|endoftext|>'

In [11]:
from transformers import AutoTokenizer

model_path = '/share/huggingface/'
model_name = 'Qwen-1_8B-Chat'
tokenizer = AutoTokenizer.from_pretrained(model_path + model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token = '<|endoftext|>'
tokenizer.pad_token_id = tokenizer.eos_token_id = 151643
prompt_tokens = tokenizer.batch_encode_plus(prompt_list, padding=True, return_tensors='pt')['input_ids']
# ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [13]:
prompt_tokens.shape

torch.Size([133, 41])

In [15]:
def tokenize_prompt_list(prompt_list):
    token_list, token_length_list = [], []
    for prompt in prompt_list:
        token_list.append(tokenizer(prompt, padding=False)['input_ids'])
        token_length_list.append(len(token_list[-1]))
    return token_list, token_length_list

token_list, token_length_list = tokenize_prompt_list(prompt_list)
sum(token_length_list) / len(token_length_list)

22.30827067669173